In [ ]:
from elasticsearch import Elasticsearch
es_client = Elasticsearch('http://localhost:9200')
#es_client = Elasticsearch('http://host.docker.internal:9200')

In [2]:
!curl http://host.docker.internal:9200

curl: (6) Could not resolve host: host.docker.internal


In [9]:
import requests

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [10]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"}
        }
    }
}

index_name = "course-questions"

es_client.indices.create(index=index_name, body=index_settings)

BadRequestError: BadRequestError(400, 'resource_already_exists_exception', 'index [course-questions/BkGnUJZuSrqRGSS745w_Bw] already exists')

In [5]:
from tqdm.auto import tqdm
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

  0%|          | 0/948 [00:00<?, ?it/s]

In [11]:
def elastic_search(query):
    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text"],
                        "type": "best_fields"
                    }
                },

            }
        }
    }

    response = es_client.search(index=index_name, body=search_query)
    print("RESP: ", response)
    result_docs = []

    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])

    return result_docs

In [12]:
query_question = 'How do execute a command on a Kubernetes pod?'

In [13]:
search_results = elastic_search(query_question)
print(search_results)

RESP:  {'took': 334, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 739, 'relation': 'eq'}, 'max_score': 44.50556, 'hits': [{'_index': 'course-questions', '_id': 'jPIHdpcBsF65hyciCAF9', '_score': 44.50556, '_source': {'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)', 'section': '5. Deploying Machine Learning Models', 'question': 'How do I debug a docker container?', 'course': 'machine-learning-zoomcamp'}}, {'_index': 'course-questions', '_id': 'G_IHdpcBsF65hyciDAI9', '_score': 35.433445, '_source': {'text': 'Deploy and Access the Kubernetes Dashboard\nLuke', 'section': '10. Kubernetes and TensorFlow Serving', 'question': 'Kubernetes-dashb

In [14]:
def elastic_search(query):
    search_query = {
        "size": 3,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "machine-learning-zoomcamp"
                    }
                }
            }
        }
    }

    response = es_client.search(index=index_name, body=search_query)

    result_docs = []

    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])

    return result_docs

In [15]:
query_question = "How do copy a file to a Docker container?"
search_results = elastic_search(query_question)
print(search_results[2]['question'])

How do I copy files from a different folder into docker container’s working directory?


In [16]:
context_template = ''

for r in search_results:
    question = r['question']
    text = r['text']

    context_template = context_template + \
    f"""
Q: {question}
A: {text}
    """.strip() + "\n\n"


In [17]:
print(context_template)

Q: How do I debug a docker container?
A: Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.
docker run -it --entrypoint bash <image>
If the container is already running, execute a command in the specific container:
docker ps (find the container-id)
docker exec -it <container-id> bash
(Marcos MJD)

Q: How do I copy files from my local machine to docker container?
A: You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:
To copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:
docker cp /path/to/local/file_or_directory container_id:/path/in/container
Hrithik Kumar Advani

Q: How do I copy files from a different folder into docker container’s working directory?
A: You can copy files from your local machine into a Docker container using the docker cp command. Here's how to 

In [18]:
prompt_template = f"""
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {query_question}

CONTEXT:
{context_template}
""".strip()

In [19]:
print(prompt_template)

You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: How do copy a file to a Docker container?

CONTEXT:
Q: How do I debug a docker container?
A: Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.
docker run -it --entrypoint bash <image>
If the container is already running, execute a command in the specific container:
docker ps (find the container-id)
docker exec -it <container-id> bash
(Marcos MJD)

Q: How do I copy files from my local machine to docker container?
A: You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:
To copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:
docker cp /path/to/local/file_or_directory container_id:/path/in/contain

In [20]:
print(len(context_template))

1220


In [11]:
import boto3
import json

import boto3.session

session = boto3.session.Session(profile_name="jose", region_name="us-west-2")

bedrock_runtime_client = session.client('bedrock-runtime')
%env AWS_PROFILE=jose
%env AWS_REGION=us-west-2


env: AWS_PROFILE=jose
env: AWS_REGION=us-west-2


In [12]:
client = session.client("sts")
print(client.get_caller_identity()["Account"])

261156800081


In [13]:
MICRO_MODEL_ID = "us.amazon.nova-micro-v1:0"
LITE_MODEL_ID = "us.amazon.nova-lite-v1:0"
PRO_MODEL_ID = "us.amazon.nova-pro-v1:0"

In [14]:
def call_nova(model,
              messages,
              system_message='',
              stream='off',
              max_tokens=1024,
              temp=0.7,
              top_p=0.99,
              top_k=20,
              tools=None,
              verbose=False):
    system_list = [{"text": system_message}]
    inf_params = {"max_new_tokens": max_tokens, "top_p": top_p, "top_k": top_k,
                  "temperature": temp}
    request_body = {"messages": messages,
                    "system": system_list,
                    "inferenceConfig": inf_params
                   }
    if tools is not None:
        tool_config=[]
        for tool in tools:
            tool_config.append({"toolSpec": tool})
        request_body['toolConfig'] = {"tools":tool_config}
    if verbose:
        print("Request Body", request_body)
    if stream=='off':
        response = bedrock_runtime_client.invoke_model(modelId=model, body=json.dumps(request_body))
        model_response = json.loads(response["body"].read())
        return model_response, model_response["output"]["message"]["content"][0]['text']
    else:
        response = bedrock_runtime_client.invoke_model_with_response_stream(modelId=model, body=json.dumps(request_body))
        return response["body"]

In [15]:
messages = [{"role": "user", "content": [{"text": prompt_template}]}]
system_message = ""
model_response, content_text = call_nova(
    model=MICRO_MODEL_ID,
    messages=messages,
    system_message=system_message
    )

print("\n[Full Response]")
print(json.dumps(model_response, indent=2))
print("\n[Response Content Text]")
print(content_text)

NameError: name 'prompt_template' is not defined

In [10]:
client = boto3.client("sts")
print(client.get_caller_identity()["Account"])

ClientError: An error occurred (InvalidClientTokenId) when calling the GetCallerIdentity operation: The security token included in the request is invalid.